In [1]:
from lmexp.models.implementations.gpt2small import GPT2Tokenizer, ProbedGPT2
from lmexp.generic.probing import train_probe
from lmexp.generic.caa import get_caa_vecs
from lmexp.generic.hooked_model import run_simple_steering
from datetime import datetime
import random

# Load model and tokenizer

These classes have already implemented all the probing-related methods so we won't have to add more hooks + they are ready to use with our vector extraction and steering functions.

In [2]:
model = ProbedGPT2()
tokenizer = GPT2Tokenizer()

In [3]:
model.get_n_layers()

12

# Training a linear probe

## Generate some data

Let's see whether we can get a date/time probe vector

In [4]:
def gen_labeled_text(n):
    # date as text, date as utc timestamp in seconds, sample randomly from between 1990 and 2022
    start_timestamp = datetime(1990, 1, 1).timestamp()
    end_timestamp = datetime(2022, 1, 1).timestamp()
    labeled_text = []
    for i in range(n):
        timestamp = start_timestamp + (end_timestamp - start_timestamp) * random.random()
        date = datetime.fromtimestamp(timestamp)
        # date like "Monday 15th November 2021 8AM"
        text = date.strftime("Today is a %A. It's the %dth of %B, %Y. The time is %I %p. This is the point in time when")
        label = timestamp
        labeled_text.append((text, label))
    # normalize labels to have mean 0 and std 1
    labels = [label for _, label in labeled_text]
    mean = sum(labels) / len(labels)
    std = (sum((label - mean) ** 2 for label in labels) / len(labels)) ** 0.5
    labeled_text = [(text, (label - mean) / std) for text, label in labeled_text]
    return labeled_text

In [5]:
data = gen_labeled_text(500)
print(data[0])

("Today is a Monday. It's the 01th of July, 1996. The time is 03 PM. This is the point in time when", -1.0468708530019906)


## Training

In [ ]:
probe = train_probe(
    labeled_text=data,
    model=model,
    tokenizer=tokenizer,
    layer=4,
    n_epochs=3,
    batch_size=16,
    lr=1e-3,
    save_to=None
)

## Using the vector

In [ ]:
direction = probe.weight[0]
bias = probe.bias

In [ ]:
results = run_simple_steering(
    text=["The current date is"],
    model=model,
    tokenizer=tokenizer,
    layer=5,
    multiplier=-2,
    vector=direction,
    max_n_tokens=10,
    save_to=None
)

In [ ]:
results

# CAA

## Let's get some contrast pairs

Let's try an easy direction - positive vs negative sentiment

In [6]:
GOOD = [
    "The weather is really nice",
    "I'm so happy",
    "This cake is absolutely delicious",
    "I love my friends",
    "I'm feeling great",
    "I'm so excited",
    "This is the best day ever",
    "I really like this gift",
    "Croissants are my favorite",
    "The movie was fantastic",
    "I got a promotion at work",
    "My vacation was amazing",
    "The concert exceeded my expectations",
    "I'm grateful for my family",
    "This book is incredibly engaging",
    "The restaurant service was excellent",
    "I'm proud of my accomplishments",
    "The sunset is breathtakingly beautiful",
    "I passed my exam with flying colors",
    "This coffee tastes perfect",
]

BAD = [
    "The weather is really bad",
    "I'm so sad",
    "This cake is completely inedible",
    "I hate my enemies",
    "I'm feeling awful",
    "I'm so anxious",
    "This is the worst day ever",
    "I dislike this gift",
    "Croissants are disgusting",
    "The movie was terrible",
    "I got fired from work",
    "My vacation was a disaster",
    "The concert was a huge disappointment",
    "I'm frustrated with my family",
    "This book is incredibly boring",
    "The restaurant service was horrible",
    "I'm ashamed of my mistakes",
    "The weather is depressingly gloomy",
    "I failed my exam miserably",
    "This coffee tastes awful",
]

In [7]:
dataset = [
    (text, True) for text in GOOD
] + [
    (text, False) for text in BAD
]

## Getting the CAA vectors

In [8]:
vectors = get_caa_vecs(
    labeled_text=dataset,
    model=model,
    tokenizer=tokenizer,
    layers=range(3, 8),
    save_to=None              
)

100%|██████████| 40/40 [00:02<00:00, 17.81it/s]


## Using the CAA vectors

In [30]:
run_simple_steering(
    text=["I think that this cat is"],
    model=model,
    tokenizer=tokenizer,
    layer=6,
    multiplier=-2,
    vector=vectors[6],
    max_n_tokens=20,
    save_to=None,
)

[{'input': 'I think that this cat is',
  'output': "I think that this cat is a bit of a mess. It's not a good cat. It",
  'layer': 6,
  'multiplier': -2}]

In [28]:
run_simple_steering(
    text=["I think that this cat is"],
    model=model,
    tokenizer=tokenizer,
    layer=6,
    multiplier=2,
    vector=vectors[6],
    max_n_tokens=20,
    save_to=None,
)

[{'input': 'I think that this cat is',
  'output': 'I think that this cat is a great addition to the collection of cats that we have in our collection',
  'layer': 6,
  'multiplier': 2}]